In [40]:
# import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances



# Load dataset
columns = ['movieID','userID','Ratings']
users_movies = pd.read_csv('exampledataset/data.txt', delimiter=',', names=columns)
users_movies.head()


# generate matrix

rating_matrix = users_movies.pivot(index='userID', columns='movieID',values='Ratings')

rating_matrix.fillna(0, inplace=True)

rating_matrix.head(35)
# item-based recommendation system



movieID,1,2,3,4,5,6
userID,,,,,,
1,1.0,0.0,2.0,0.0,0.0,1.0
2,0.0,0.0,4.0,2.0,0.0,0.0
3,3.0,5.0,0.0,4.0,4.0,3.0
4,0.0,4.0,1.0,0.0,3.0,0.0
5,0.0,0.0,2.0,5.0,4.0,3.0
6,5.0,0.0,0.0,0.0,2.0,0.0
7,0.0,4.0,3.0,0.0,0.0,0.0
8,0.0,0.0,0.0,4.0,0.0,2.0
9,5.0,0.0,4.0,0.0,0.0,0.0


In [41]:
def findSimilarItemsForAnItem(item, dataset,n_neighbors):
    similarities = []
    indices = []
    dataset = dataset.T
    item_location = dataset.index.get_loc(item)
    
    knn_model = NearestNeighbors(metric='cosine',algorithm='brute', n_neighbors=n_neighbors)
    knn_model.fit(dataset)
    distances, indices = knn_model.kneighbors(dataset.iloc[item_location,:].values.reshape(1,-1), n_neighbors=n_neighbors)
    similarities = 1-distances.flatten();
    return similarities, indices


In [52]:
def predictItemsToUser(user_id, item_id, dataset, n_neighbors):
    prediction = 0
    weighted_sum = 0
    user_location = dataset.index.get_loc(user_id)
    item_location = dataset.columns.get_loc(item_id);
    similarities, indices = findSimilarItemsForAnItem(item_id,dataset,5)
    sum_weight = np.sum(similarities)-1
    
    product = 1
    
    for i in range(len(indices.flatten())):
        if (i!=item_location):
            
            product = dataset.iloc[user_location, indices.flatten()[i]]*(similarities[i])
            weighted_sum = weighted_sum + product
        prediction = weighted_sum/sum_weight
    
        prediction = 0 if prediction <=0 else prediction
        prediction = 5 if prediction > 5 else prediction
        print('\nPredicting Rating of User {0} for item {1}: {2}'.format(user_id, item_id, prediction))
        return prediction

prediction = predictItemsToUser(12,5,rating_matrix,2)


Predicting Rating of User 12 for item 5: 2.2237474513959277
